# This notebook contains a simple introduction to TF
### as used in:

* <font size=4 color='green'>MSTC seminar on Deep Learning, Tensorflow & Keras</font>

In [0]:
! pip show tensorflow

In [0]:
#! pip install --upgrade tensorflow

In [0]:
import numpy as np

x=np.array([1.0])
W=np.array([6.0])
b=np.array([1.5])

y=x*W+b
print(y)

In [0]:
import tensorflow as tf

x=tf.constant(1.0)
W=tf.constant(6.0)
b=tf.constant(1.5)

y=x*W+b

print(y)

In [0]:
with tf.Session() as sess:
    print(sess.run(y))

<font size=5 color=F000AA>HOWEVER *Tensorflow* is open to new venues!</font>

## See: Eager Execution for TensorFlow
Announcing TensorFlow 1.5
Friday, January 26, 2018

https://developers.googleblog.com/2018/01/announcing-tensorflow-15.html


https://docs.google.com/presentation/d/1_1CFkL4IDqWtTM_pv4U-Tr9sEiFMTMpWJSzts8YKtxo/edit#slide=id.g26499ba2c0_0_246

### Now let's see a simple use of Variables in Tensorflow

In [0]:
x=tf.constant(1.0)
W=tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b=tf.Variable(tf.zeros([1]))

# Before starting, initialize the variables
# new versions of tf use: 
init = tf.global_variables_initializer()
#init = tf.initialize_all_variables()

y=x*W+b

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(y))


sess.close()

In [0]:
# Create an Interactive session
sess=tf.InteractiveSession()

x=tf.constant(1.0)
W=tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b=tf.Variable(tf.zeros([1]))

# Before starting, initialize the variables
# new versions of tf use: 
init = tf.global_variables_initializer()
#init = tf.initialize_all_variables()

y=x*W+b

sess.run(init)

#just call eval() on the computation to be evaluated.
print(y.eval())

sess.close()


# Placeholders: computational graph entry points

In [0]:
x=tf.placeholder(tf.float32)
W=tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b=tf.Variable(tf.zeros([1]))

# Before starting, initialize the variables
# new versions of tf use: 
init = tf.global_variables_initializer()
#init = tf.initialize_all_variables()

# Define and Operation with Tensors
y=x*W+b


with tf.Session() as sess:
    sess.run(init)
    for step in range(4):
        print(sess.run(y,feed_dict={x:6.0}))
        print(sess.run(y,feed_dict={x:np.random.rand(1)}))

## Using TensorFlow tensors opperations: tf.add  and   tf.matmul

In [0]:

x=tf.placeholder(tf.float32,[1,1])
W=tf.Variable(tf.random_uniform([1,1], -1.0, 1.0))
b=tf.Variable(tf.zeros([1]))

# Before starting, initialize the variables
# new versions of tf use: 
init = tf.global_variables_initializer()
#init = tf.initialize_all_variables()

# Define and Operation with Tensors
#y=x*W+b
y=tf.add(tf.matmul(x,W),b)


with tf.Session() as sess:
    sess.run(init)
    for step in range(4):
        zz=np.array([6]).reshape(1,1)
        print(sess.run(y,feed_dict={x:zz}))
        
sess.close()


# Print the Graph : Better use [TensorBoard](https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard)

In [0]:
print(tf.get_default_graph().as_graph_def())

# Finally: extracting more data from $Graphs$

In [100]:
sess=tf.Session()

sess.run(init)

zz=np.array([6]).reshape(1,1)
yFetch, WFetch=sess.run([y,W],feed_dict={x:zz})

print("y value",yFetch, "W value", WFetch)


sess.close()

y value [[5.1469746]] W value [[0.8578291]]


In [101]:
zz*WFetch

array([[5.14697456]])